# Bereinigung der Daten

In [66]:
#Import Bibliothek
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from langdetect import detect, LangDetectException
from collections import Counter
import re
from googletrans import Translator
import math

In [67]:
#1. Daten nach Datensammlung importieren
file_path = r'5.1.1_Datensammlung\4_Daten_alle.csv'

df = pd.read_csv(file_path)

#Ergänzung Anzahl der Wörter (siehe Datenverständnis)
df['Anzahl Woerter Text'] = df['Text'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
total_words_text = df['Anzahl Woerter Text'].sum()
print("Gesamtanzahl der Woerter im Text:", total_words_text)

#Prüfung der Daten
print(df.shape)
print(df.describe())
print(df.head())



Gesamtanzahl der Woerter im Text: 5204197
(10841, 6)
       Anzahl Woerter Text
count         10841.000000
mean            480.047689
std             493.846364
min               0.000000
25%             109.000000
50%             376.000000
75%             675.000000
max            5771.000000
  Quelle       Datum                                               Link  \
0    FAZ  11/30/2023  https://www.faz.net/aktuell/wirtschaft/unterne...   
1    FAZ  11/30/2023  https://www.faz.net/aktuell/wirtschaft/kuenstl...   
2    FAZ  11/29/2023  https://www.faz.net/aktuell/feuilleton/medien/...   
3    FAZ  11/28/2023  https://www.faz.net/pro/d-economy/kuenstliche-...   
4    FAZ  11/28/2023  https://www.faz.net/pro/d-economy/kuenstliche-...   

                                               Titel  \
0  KI: Warum wir nicht mit Roboter-Autos vollauto...   
1  Microsoft: Sind Jahrzehnte von einer künstlich...   
2  Künstliche Intelligenz: Europa muss von neuen ...   
3  Bilderstrecke zu: Was die 

In [68]:
#Datum in Datumsformat umwandeln
df['Datum'] = pd.to_datetime(df['Datum'], format='%m/%d/%Y', errors='coerce')

# Entfernung von irrelevanten Nachrichtenartikel

In [69]:
#Leere Datensaetze entfernen
df = df.dropna(subset=['Datum'])
df = df.dropna(subset=['Titel'])
df = df.dropna(subset=['Text'])

#Prüfung der Daten
print(df.shape)

(9562, 6)


In [70]:
#Entfernen aller Duplikate
df = df.drop_duplicates()

#Pruefung der Daten
print(df.shape)

(5440, 6)


In [71]:
#Artikel mit weniger als 250 Woerter entfernen
df = df[(df['Anzahl Woerter Text'] >= 250)]

#Prüfung der Daten
print(df.shape)

(3936, 6)


In [72]:
#Artikel in englischer Sprache entfernen 

def is_english(content):
    try:
        if pd.notnull(content) and isinstance(content, str):
            return detect(content) == 'en'
    except LangDetectException:
        return False
    return False

#Wende die Funktion auf die Spalte 'Content' an und erhalte eine Serie, die angibt, ob der Artikel englisch ist
df['is_english'] = df['Text'].apply(is_english)

#Filtere den DataFrame, um nur Zeilen zu behalten, die nicht englisch sind
df = df[~df['is_english']]

#Entferne die zusätzliche Hilfsspalte 'is_english'
df.drop(columns=['is_english'], inplace=True)

#Pruefung der Daten
print(df.shape)

(3903, 6)


In [73]:
#Manuelle Entfernen von Zeilen, welche als nicht Nachrichtenartikel identifiziert wurden (z.B. Newsticker, Videos, Bildergallerien, Briefe)

#Definiere Titel mit unrelevante Nachrichtenartikel
condition = df['Titel'].str.contains('News des Tages|Briefe - DER SPIEGEL|Leserbriefe|Nachrichten vom', regex=True, case=False) 

#Filtere den DataFrame, um nur Zeilen zu behalten, die NICHT den Bedingungen entsprechen
df = df[~condition]

#Pruefung der Daten
print(df.shape)

(3874, 6)


# Uebersetzung der Ueberschriften und Textinhalte auf Englisch

In [74]:
#Translator-Objekt erstellen
translator = Translator()

#Sichere Übersetzungsfunktion mit Fehlerbehandlung
def safe_translate(text, src, dest):
    try:
        translation = translator.translate(text, src=src, dest=dest)
        if translation:
            return translation.text
        else:
            return ""
    except Exception as e:
        print(f"Übersetzungsfehler: {e}")
        return ""

In [75]:
#Funktion zum Teilen des Textes in 500-Wort-Chunks
def split_text(text):
    words = text.split()
    return [ ' '.join(words[i:i+500]) for i in range(0, len(words), 500) ]

In [76]:
#Hauptschleife zur Bearbeitung des DataFrame
def process_dataframe(df):
    df['Text'] = df['Text'].astype(str)
    
    total_rows = len(df)
    for index, row in df.iterrows():
        split_texts = split_text(row['Text'])
        translated_texts = []
        
        for text_chunk in split_texts:
            translated_texts.append(safe_translate(text_chunk, src='de', dest='en'))
        
        #Zusammenführen der übersetzten Teile
        df.at[index, 'Text_bereinigt'] = ' '.join(translated_texts)
        
        #Statusanzeige
        print(f"Bearbeitet: {index + 1}/{total_rows} ({(index + 1) / total_rows * 100:.2f}%)")
    
    return df

In [77]:
#DataFrame verarbeiten
processed_df = process_dataframe(df)

Bearbeitet: 1/3874 (0.03%)
Bearbeitet: 2/3874 (0.05%)
Bearbeitet: 3/3874 (0.08%)
Bearbeitet: 5/3874 (0.13%)
Bearbeitet: 6/3874 (0.15%)
Bearbeitet: 7/3874 (0.18%)
Bearbeitet: 10/3874 (0.26%)
Bearbeitet: 11/3874 (0.28%)
Bearbeitet: 12/3874 (0.31%)
Bearbeitet: 20/3874 (0.52%)
Bearbeitet: 23/3874 (0.59%)
Bearbeitet: 24/3874 (0.62%)
Bearbeitet: 25/3874 (0.65%)
Bearbeitet: 26/3874 (0.67%)
Bearbeitet: 28/3874 (0.72%)
Bearbeitet: 31/3874 (0.80%)
Bearbeitet: 33/3874 (0.85%)
Bearbeitet: 34/3874 (0.88%)
Bearbeitet: 35/3874 (0.90%)
Bearbeitet: 37/3874 (0.96%)
Bearbeitet: 40/3874 (1.03%)
Bearbeitet: 41/3874 (1.06%)
Bearbeitet: 45/3874 (1.16%)
Bearbeitet: 48/3874 (1.24%)
Bearbeitet: 50/3874 (1.29%)
Bearbeitet: 51/3874 (1.32%)
Bearbeitet: 53/3874 (1.37%)
Bearbeitet: 57/3874 (1.47%)
Bearbeitet: 58/3874 (1.50%)
Bearbeitet: 59/3874 (1.52%)
Bearbeitet: 61/3874 (1.57%)
Bearbeitet: 63/3874 (1.63%)
Bearbeitet: 64/3874 (1.65%)
Bearbeitet: 65/3874 (1.68%)
Bearbeitet: 68/3874 (1.76%)
Bearbeitet: 69/3874 (1.78%

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 440/3874 (11.36%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 441/3874 (11.38%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 445/3874 (11.49%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 447/3874 (11.54%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 448/3874 (11.56%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Tran

Bearbeitet: 748/3874 (19.31%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 751/3874 (19.39%)
Bearbeitet: 753/3874 (19.44%)
Bearbeitet: 754/3874 (19.46%)
Bearbeitet: 755/3874 (19.49%)
Bearbeitet: 756/3874 (19.51%)
Bearbeitet: 757/3874 (19.54%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 759/3874 (19.59%)
Bearbeitet: 766/3874 (19.77%)
Bearbeitet: 773/3874 (19.95%)
Bearbeitet: 775/3874 (20.01%)
Bearbeitet: 776/3874 (20.03%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 777/3874 (20.06%)
Bearbeitet: 778/3874 (20.08%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 780/3874 (20.13%)
Bearbeitet: 781/3874 (20.16%)
Bearbeitet: 784/3874 (20.24%)
Bearbeitet: 793/3874 (20.47%)
Bearbeitet: 795/3874 (20.52%)
Bearbeitet: 803/3874 (20.73%)
Bearbeitet: 806/3874 (20.81%)
Bearbeitet: 807/3874 (20.83%)
Bearbeitet: 809/3874 (20.88%)
Übersetzun

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1095/3874 (28.27%)
Bearbeitet: 1102/3874 (28.45%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1104/3874 (28.50%)
Bearbeitet: 1108/3874 (28.60%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1109/3874 (28.63%)
Bearbeitet: 1113/3874 (28.73%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1120/3874 (28.91%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1121/3874 (28.94%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Be

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1434/3874 (37.02%)
Bearbeitet: 1435/3874 (37.04%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1440/3874 (37.17%)
Bearbeitet: 1441/3874 (37.20%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1442/3874 (37.22%)
Bearbeitet: 1445/3874 (37.30%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1448/3874 (37.38%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1449/3874 (37.40%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1451/3874 (37.45%)
Bearbeitet: 1458/3874 (37.64%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1459/3874 (37.66%)
Übersetzungsfehler: 'Translator' object has no attribute 'r

...

Bearbeitet: 8834/3874 (228.03%)
Bearbeitet: 8835/3874 (228.06%)
Bearbeitet: 8838/3874 (228.14%)
Bearbeitet: 8839/3874 (228.16%)
Bearbeitet: 8842/3874 (228.24%)
Bearbeitet: 8843/3874 (228.27%)
Bearbeitet: 8844/3874 (228.29%)
Bearbeitet: 8845/3874 (228.32%)
Bearbeitet: 8848/3874 (228.39%)
Bearbeitet: 8849/3874 (228.42%)
Bearbeitet: 8850/3874 (228.45%)
Bearbeitet: 8851/3874 (228.47%)
Bearbeitet: 8853/3874 (228.52%)
Bearbeitet: 8861/3874 (228.73%)
Bearbeitet: 8862/3874 (228.76%)
Bearbeitet: 8863/3874 (228.78%)
Bearbeitet: 8864/3874 (228.81%)
Bearbeitet: 8865/3874 (228.83%)
Bearbeitet: 8866/3874 (228.86%)
Bearbeitet: 8867/3874 (228.88%)
Bearbeitet: 8868/3874 (228.91%)
Bearbeitet: 8871/3874 (228.99%)
Bearbeitet: 8873/3874 (229.04%)
Bearbeitet: 8876/3874 (229.12%)
Bearbeitet: 8877/3874 (229.14%)
Bearbeitet: 8878/3874 (229.17%)
Bearbeitet: 8879/3874 (229.19%)
Bearbeitet: 8882/3874 (229.27%)
Bearbeitet: 8886/3874 (229.38%)
Bearbeitet: 8887/3874 (229.40%)
Bearbeitet: 8888/3874 (229.43%)
Bearbeit

Bearbeitet: 9194/3874 (237.33%)
Bearbeitet: 9195/3874 (237.35%)
Bearbeitet: 9196/3874 (237.38%)
Bearbeitet: 9197/3874 (237.40%)
Bearbeitet: 9199/3874 (237.45%)
Bearbeitet: 9206/3874 (237.64%)
Bearbeitet: 9207/3874 (237.66%)
Bearbeitet: 9208/3874 (237.69%)
Bearbeitet: 9209/3874 (237.71%)
Bearbeitet: 9211/3874 (237.76%)
Bearbeitet: 9216/3874 (237.89%)
Bearbeitet: 9219/3874 (237.97%)
Bearbeitet: 9220/3874 (238.00%)
Bearbeitet: 9221/3874 (238.02%)
Bearbeitet: 9224/3874 (238.10%)
Bearbeitet: 9230/3874 (238.26%)
Bearbeitet: 9231/3874 (238.28%)
Bearbeitet: 9232/3874 (238.31%)
Bearbeitet: 9233/3874 (238.33%)
Bearbeitet: 9237/3874 (238.44%)
Bearbeitet: 9242/3874 (238.56%)
Bearbeitet: 9245/3874 (238.64%)
Bearbeitet: 9247/3874 (238.69%)
Bearbeitet: 9248/3874 (238.72%)
Bearbeitet: 9249/3874 (238.75%)
Bearbeitet: 9254/3874 (238.87%)
Bearbeitet: 9255/3874 (238.90%)
Bearbeitet: 9256/3874 (238.93%)
Bearbeitet: 9257/3874 (238.95%)
Bearbeitet: 9258/3874 (238.98%)
Bearbeitet: 9264/3874 (239.13%)
Bearbeit

Bearbeitet: 9607/3874 (247.99%)
Bearbeitet: 9608/3874 (248.01%)
Bearbeitet: 9610/3874 (248.06%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9616/3874 (248.22%)
Bearbeitet: 9617/3874 (248.24%)
Bearbeitet: 9621/3874 (248.35%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9625/3874 (248.45%)
Bearbeitet: 9626/3874 (248.48%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9627/3874 (248.50%)
Bearbeitet: 9628/3874 (248.53%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9635/3874 (248.71%)
Bearbeitet: 9637/3874 (248.76%)
Bearbeitet: 9638/3874 (248.79%)
Bearbeitet: 9642/3874 (248.89%)
Bearbeitet: 9643/3874 (248.92%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9656/3874 (249.25%)
Bearbeitet: 9657/3874 (249.28%)
Bearbe

Bearbeitet: 10370/3874 (267.68%)
Bearbeitet: 10371/3874 (267.71%)
Bearbeitet: 10372/3874 (267.73%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 10374/3874 (267.79%)
Bearbeitet: 10375/3874 (267.81%)
Bearbeitet: 10376/3874 (267.84%)
Bearbeitet: 10377/3874 (267.86%)
Bearbeitet: 10378/3874 (267.89%)
Bearbeitet: 10379/3874 (267.91%)
Bearbeitet: 10380/3874 (267.94%)
Bearbeitet: 10381/3874 (267.97%)
Bearbeitet: 10384/3874 (268.04%)
Bearbeitet: 10385/3874 (268.07%)
Bearbeitet: 10386/3874 (268.09%)
Bearbeitet: 10387/3874 (268.12%)
Bearbeitet: 10388/3874 (268.15%)
Bearbeitet: 10390/3874 (268.20%)
Bearbeitet: 10392/3874 (268.25%)
Bearbeitet: 10393/3874 (268.28%)
Bearbeitet: 10394/3874 (268.30%)
Bearbeitet: 10395/3874 (268.33%)
Bearbeitet: 10396/3874 (268.35%)
Bearbeitet: 10397/3874 (268.38%)
Bearbeitet: 10398/3874 (268.40%)
Bearbeitet: 10399/3874 (268.43%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 10400/3874 (2

Bearbeitet: 10816/3874 (279.19%)
Bearbeitet: 10817/3874 (279.22%)
Bearbeitet: 10820/3874 (279.30%)
Bearbeitet: 10822/3874 (279.35%)
Bearbeitet: 10826/3874 (279.45%)
Bearbeitet: 10829/3874 (279.53%)
Bearbeitet: 10830/3874 (279.56%)
Bearbeitet: 10831/3874 (279.58%)
Bearbeitet: 10833/3874 (279.63%)
Bearbeitet: 10840/3874 (279.81%)


In [78]:
#Hauptschleife zur Bearbeitung des DataFrame
def process_dataframe(df):
    df['Titel'] = df['Titel'].astype(str)
    
    total_rows = len(df)
    for index, row in df.iterrows():
        split_texts = split_text(row['Titel'])
        translated_texts = []
        
        for text_chunk in split_texts:
            translated_texts.append(safe_translate(text_chunk, src='de', dest='en'))
        
        #Zusammenführen der übersetzten Teile
        df.at[index, 'Titel_bereinigt'] = ' '.join(translated_texts)
        
        #Statusanzeige
        print(f"Bearbeitet: {index + 1}/{total_rows} ({(index + 1) / total_rows * 100:.2f}%)")
    
    return df

In [79]:
#DataFrame verarbeiten
processed_df = process_dataframe(df)

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1/3874 (0.03%)
Bearbeitet: 2/3874 (0.05%)
Bearbeitet: 3/3874 (0.08%)
Bearbeitet: 5/3874 (0.13%)
Bearbeitet: 6/3874 (0.15%)
Bearbeitet: 7/3874 (0.18%)
Bearbeitet: 10/3874 (0.26%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 11/3874 (0.28%)
Bearbeitet: 12/3874 (0.31%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 20/3874 (0.52%)
Bearbeitet: 23/3874 (0.59%)
Bearbeitet: 24/3874 (0.62%)
Bearbeitet: 25/3874 (0.65%)
Bearbeitet: 26/3874 (0.67%)
Bearbeitet: 28/3874 (0.72%)
Bearbeitet: 31/3874 (0.80%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 33/3874 (0.85%)
Bearbeitet: 34/3874 (0.88%)
Bearbeitet: 35/3874 (0.90%)
Bearbeitet: 37/3874 (0.96%)
Bearbeitet: 40/3874 (1.03%)
Bearbeitet: 41/3874 (1.06%)
Bearbeitet: 45/3874 (1.16%)
Bearbeitet: 48/3874 (1.24%)
Bearbeitet: 50/3874 (1.29%)
Überse

Bearbeitet: 511/3874 (13.19%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 512/3874 (13.22%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 517/3874 (13.35%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 528/3874 (13.63%)
Bearbeitet: 531/3874 (13.71%)
Bearbeitet: 532/3874 (13.73%)
Bearbeitet: 533/3874 (13.76%)
Bearbeitet: 534/3874 (13.78%)
Bearbeitet: 536/3874 (13.84%)
Bearbeitet: 539/3874 (13.91%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 544/3874 (14.04%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 545/3874 (14.07%)
Bearbeitet: 547/3874 (14.12%)
Bearbeitet: 552/3874 (14.25%)
Bearbeitet: 556/3874 (14.35%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 558/3874 (14.40%)
Bearbeitet: 562/3874 (14.51%)
Bearbeitet: 563/3874 (14.53%)
Bearbeitet

Bearbeitet: 1055/3874 (27.23%)
Bearbeitet: 1056/3874 (27.26%)
Bearbeitet: 1057/3874 (27.28%)
Bearbeitet: 1060/3874 (27.36%)
Bearbeitet: 1061/3874 (27.39%)
Bearbeitet: 1063/3874 (27.44%)
Bearbeitet: 1067/3874 (27.54%)
Bearbeitet: 1071/3874 (27.65%)
Bearbeitet: 1073/3874 (27.70%)
Bearbeitet: 1078/3874 (27.83%)
Bearbeitet: 1079/3874 (27.85%)
Bearbeitet: 1081/3874 (27.90%)
Bearbeitet: 1085/3874 (28.01%)
Bearbeitet: 1089/3874 (28.11%)
Bearbeitet: 1090/3874 (28.14%)
Bearbeitet: 1091/3874 (28.16%)
Bearbeitet: 1092/3874 (28.19%)
Bearbeitet: 1095/3874 (28.27%)
Bearbeitet: 1102/3874 (28.45%)
Bearbeitet: 1104/3874 (28.50%)
Bearbeitet: 1108/3874 (28.60%)
Bearbeitet: 1109/3874 (28.63%)
Bearbeitet: 1113/3874 (28.73%)
Bearbeitet: 1120/3874 (28.91%)
Bearbeitet: 1121/3874 (28.94%)
Bearbeitet: 1123/3874 (28.99%)
Bearbeitet: 1129/3874 (29.14%)
Bearbeitet: 1133/3874 (29.25%)
Bearbeitet: 1140/3874 (29.43%)
Bearbeitet: 1141/3874 (29.45%)
Bearbeitet: 1142/3874 (29.48%)
Bearbeitet: 1144/3874 (29.53%)
Bearbeit

Bearbeitet: 1642/3874 (42.39%)
Bearbeitet: 1644/3874 (42.44%)
Bearbeitet: 1646/3874 (42.49%)
Bearbeitet: 1647/3874 (42.51%)
Bearbeitet: 1650/3874 (42.59%)
Bearbeitet: 1651/3874 (42.62%)
Bearbeitet: 1653/3874 (42.67%)
Bearbeitet: 1654/3874 (42.69%)
Bearbeitet: 1656/3874 (42.75%)
Bearbeitet: 1657/3874 (42.77%)
Bearbeitet: 1658/3874 (42.80%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1659/3874 (42.82%)
Bearbeitet: 1660/3874 (42.85%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1661/3874 (42.88%)
Bearbeitet: 1663/3874 (42.93%)
Bearbeitet: 1665/3874 (42.98%)
Bearbeitet: 1667/3874 (43.03%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1668/3874 (43.06%)
Bearbeitet: 1669/3874 (43.08%)
Bearbeitet: 1670/3874 (43.11%)
Bearbeitet: 1671/3874 (43.13%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1672/3874 (43.16%)
Bearbeitet: 1673/3

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1880/3874 (48.53%)
Bearbeitet: 1884/3874 (48.63%)
Bearbeitet: 1886/3874 (48.68%)
Bearbeitet: 1887/3874 (48.71%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1888/3874 (48.74%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1889/3874 (48.76%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1891/3874 (48.81%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1892/3874 (48.84%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1893/3874 (48.86%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1895/3874 (48.92%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 1897/3874 (48.97%)
Übersetzungsfehler: 'Translator' object has no attribute 'r

...

Bearbeitet: 8805/3874 (227.28%)
Bearbeitet: 8806/3874 (227.31%)
Bearbeitet: 8809/3874 (227.39%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 8810/3874 (227.41%)
Bearbeitet: 8811/3874 (227.44%)
Bearbeitet: 8813/3874 (227.49%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 8814/3874 (227.52%)
Bearbeitet: 8815/3874 (227.54%)
Bearbeitet: 8816/3874 (227.57%)
Bearbeitet: 8819/3874 (227.65%)
Bearbeitet: 8820/3874 (227.67%)
Bearbeitet: 8821/3874 (227.70%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 8827/3874 (227.85%)
Bearbeitet: 8829/3874 (227.90%)
Bearbeitet: 8831/3874 (227.96%)
Bearbeitet: 8834/3874 (228.03%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 8835/3874 (228.06%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 8838/3874 (228.14%)
Bearbeitet: 8839/3874 (228.16%)
Bearbeitet: 8842/

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9055/3874 (233.74%)
Bearbeitet: 9056/3874 (233.76%)
Bearbeitet: 9057/3874 (233.79%)
Bearbeitet: 9064/3874 (233.97%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9065/3874 (234.00%)
Bearbeitet: 9066/3874 (234.02%)
Bearbeitet: 9067/3874 (234.05%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9072/3874 (234.18%)
Bearbeitet: 9073/3874 (234.20%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9077/3874 (234.31%)
Bearbeitet: 9078/3874 (234.33%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9079/3874 (234.36%)
Bearbeitet: 9080/3874 (234.38%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9081/3874 (234.41%)
Bearbeitet: 9082/3874 (234.43%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Except

Bearbeitet: 9405/3874 (242.77%)
Bearbeitet: 9410/3874 (242.90%)
Bearbeitet: 9411/3874 (242.93%)
Bearbeitet: 9412/3874 (242.95%)
Bearbeitet: 9413/3874 (242.98%)
Bearbeitet: 9414/3874 (243.00%)
Bearbeitet: 9417/3874 (243.08%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9418/3874 (243.11%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9419/3874 (243.13%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9424/3874 (243.26%)
Bearbeitet: 9425/3874 (243.29%)
Bearbeitet: 9426/3874 (243.31%)
Bearbeitet: 9429/3874 (243.39%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9434/3874 (243.52%)
Bearbeitet: 9435/3874 (243.55%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9436/3874 (243.57%)
Bearbeitet: 9437/3874 (243.60%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbe

Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9908/3874 (255.76%)
Bearbeitet: 9909/3874 (255.78%)
Bearbeitet: 9911/3874 (255.83%)
Bearbeitet: 9920/3874 (256.07%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9934/3874 (256.43%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9936/3874 (256.48%)
Bearbeitet: 9940/3874 (256.58%)
Bearbeitet: 9941/3874 (256.61%)
Bearbeitet: 9944/3874 (256.69%)
Bearbeitet: 9949/3874 (256.81%)
Bearbeitet: 9951/3874 (256.87%)
Bearbeitet: 9953/3874 (256.92%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 9955/3874 (256.97%)
Bearbeitet: 9957/3874 (257.02%)
Bearbeitet: 9960/3874 (257.10%)
Bearbeitet: 9962/3874 (257.15%)
Bearbeitet: 9973/3874 (257.43%)
Bearbeitet: 9975/3874 (257.49%)
Bearbeitet: 9977/3874 (257.54%)
Bearbeitet: 9978/3874 (257.56%)
Bearbeitet: 9980/3874 (257.61%)
Bearbeitet: 9993/3874 (257.9

Bearbeitet: 10455/3874 (269.88%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 10458/3874 (269.95%)
Bearbeitet: 10460/3874 (270.01%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 10461/3874 (270.03%)
Bearbeitet: 10462/3874 (270.06%)
Bearbeitet: 10465/3874 (270.13%)
Bearbeitet: 10469/3874 (270.24%)
Bearbeitet: 10471/3874 (270.29%)
Bearbeitet: 10472/3874 (270.31%)
Bearbeitet: 10473/3874 (270.34%)
Bearbeitet: 10474/3874 (270.37%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 10478/3874 (270.47%)
Bearbeitet: 10481/3874 (270.55%)
Bearbeitet: 10484/3874 (270.62%)
Bearbeitet: 10485/3874 (270.65%)
Bearbeitet: 10486/3874 (270.68%)
Übersetzungsfehler: 'Translator' object has no attribute 'raise_Exception'
Bearbeitet: 10487/3874 (270.70%)
Bearbeitet: 10489/3874 (270.75%)
Bearbeitet: 10490/3874 (270.78%)
Bearbeitet: 10491/3874 (270.81%)
Bearbeitet: 10493/3874 (270.86%)
Bearbei

# Entfernung von nicht relevanten Zeichen in den Ueberschriften und Textinhalten

In [80]:
#Manuelle Entfernung von nicht relevanten Inhalten

df['Text_bereinigt'] = df['Text_bereinigt'].str.lstrip(':')

common_replacements = ['--', '>', '<', '|', '»', '«', '#NAME?', '\\*',  'target=" _', 'target=', 'blank', 
    'â€™', 'Â©', 'href="https', 'href=', 'a href="', ' _blank"', '/a', '/b', '_', '/i',
    '\\\\ https', '\\\\ \\\\']

df["Titel_bereinigt"] = df['Titel_bereinigt'].replace(common_replacements,'',regex=True)
df["Text_bereinigt"] = df['Text_bereinigt'].replace(common_replacements,'',regex=True)

df["Titel_bereinigt"] = df['Titel_bereinigt'].replace(['-'], ' ', regex=True)
df["Text_bereinigt"] = df['Text_bereinigt'].replace(['-'], ' ', regex=True)

#Entfernung von "-"
df['Text_bereinigt'] = df['Text_bereinigt'].str.replace('-', ' ', regex=True)
df['Text_bereinigt'] = df['Text_bereinigt'].str.replace('-', ' ', regex=True)

# Bereinigungsmethoden zur Entfernung von Stoppwörter und Lemmatisierung

In [81]:
#Entfernen von Stoppwoertern aus Ueberschrift und Inhalt

#Stoppwoerter und Tokenizer herunterladen
nltk.download('stopwords')
nltk.download('punkt')

#Stoppwoerter laden
stop_words = set(stopwords.words('english'))

#Entfernen von Stoppwoertern
def remove_stop_words(text):
    #Tokenisierung des Textes
    word_tokens = word_tokenize(text)
    #Die Stoppwörter herausfiltern
    filtered_sentence = [word for word in word_tokens if not word.lower() in stop_words]
    #Rekonstruieren Sie den Text ohne Stoppwörter
    return ' '.join(filtered_sentence)

#Stoppwoerter aus den Spalten "UEberschrift" und "Inhalt" entfernen
df['Titel_bereinigt'] = df['Titel_bereinigt'].apply(lambda x: remove_stop_words(x) if isinstance(x, str) else x)
df['Text_bereinigt'] = df['Text_bereinigt'].apply(lambda x: remove_stop_words(x) if isinstance(x, str) else x)

print('Stoppwoerter: ', stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MarkE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MarkE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Stoppwoerter:  {'are', 'their', 't', 'don', 'my', 'such', 'hasn', 'its', "wasn't", 'does', 'there', "it's", 'didn', 'through', 'and', 'ourselves', 'after', 's', 'mustn', 'whom', "aren't", 'which', "mustn't", "mightn't", 've', 'they', 'more', 'me', 'is', 'will', 'couldn', 'm', "you've", 'herself', "that'll", 'own', 'because', 'weren', 'up', 'o', 'in', 'doing', "should've", 'll', 'aren', 'now', "you'd", "didn't", 'no', "she's", "isn't", 'doesn', 'both', 'above', 'these', 'them', 'had', 'each', 'have', 're', 'nor', 'y', 'ma', 'shan', 'again', 'for', 'than', 'an', 'until', 'other', 'then', 'off', 'this', 'most', 'by', 'did', 'so', 'been', 'while', 'i', 'has', 'below', "haven't", "couldn't", 'to', 'why', 'can', 'down', 'him', 'as', 'who', 'or', "hadn't", 'his', 'here', 'not', 'further', 'yours', 'itself', 'under', 'ours', 'am', 'too', 'being', "doesn't", 'few', 'be', 'if', 'your', "won't", 'with', 'themselves', "shouldn't", 'all', 'only', "you're", 'yourselves', 'wouldn', 'yourself', 'on', 

In [82]:
#Lemmatizierung (reduziert Wörter auf ihre Grundform)

#Sprachmodell laden
nlp = spacy.load("en_core_web_sm")

#Funktion zur Lemmatisierung einer Spalte von Textdaten
def lemmatize_text(texts):
    lemmatized_texts = []
    for text in texts:
        #Konvertierung von Nicht-String-Eingaben in Strings und Überspringen von NaN-Werten
        if pd.isnull(text):
            lemmatized_texts.append("")
        else:
            doc = nlp(str(text))
            lemmatized_text = " ".join([token.lemma_ for token in doc])
            lemmatized_texts.append(lemmatized_text)
    return lemmatized_texts

#Lemmatisierung auf die angegebenen Spalten anwenden
df["Titel_bereinigt"] = lemmatize_text(df["Titel_bereinigt"])
df["Text_bereinigt"] = lemmatize_text(df["Text_bereinigt"])

print(df.head())

  Quelle      Datum                                               Link  \
0    FAZ 2023-11-30  https://www.faz.net/aktuell/wirtschaft/unterne...   
1    FAZ 2023-11-30  https://www.faz.net/aktuell/wirtschaft/kuenstl...   
2    FAZ 2023-11-29  https://www.faz.net/aktuell/feuilleton/medien/...   
4    FAZ 2023-11-28  https://www.faz.net/pro/d-economy/kuenstliche-...   
5    FAZ 2023-11-28  https://www.faz.net/pro/d-economy/prompt-der-w...   

                                               Titel  \
0  KI: Warum wir nicht mit Roboter-Autos vollauto...   
1  Microsoft: Sind Jahrzehnte von einer künstlich...   
2  Künstliche Intelligenz: Europa muss von neuen ...   
4  Was die Superintelligenz-KI anrichten könnte, ...   
5  Künstliche Intelligenz: Wie man sich seine Pro...   

                                                Text  Anzahl Woerter Text  \
0  Roboterautos faszinieren viele – die Augen der...                 1963   
1  Der Krieg von Mensch gegen Maschine verschiebt...            

# Manuelle Entfernung von irrelevanten Zeichen in den Überschriften und Textinhalten nach der Datenbereinigung

In [96]:
#Sonderzeichen entfernen
df['Titel_bereinigt'] = df['Titel_bereinigt'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df['Text_bereinigt'] = df['Text_bereinigt'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#Entfernung doppelte Leerzeichen
df['Titel_bereinigt'] = df['Titel_bereinigt'].replace(['  '], ' ', regex=True)
df['Text_bereinigt'] = df['Text_bereinigt'].replace(['  '], ' ', regex=True)

#Ersetzen von Buchstaben, die nicht erkannt werden
df['Titel_bereinigt'] = df['Titel_bereinigt'].replace({'ü': 'ue','Ü': 'Ue', 'ö': 'oe','Ö': 'Oe', 'ä': 'ae','Ä': 'Ae', 'ß': 'ss'}, regex=True)
df['Text_bereinigt'] = df['Text_bereinigt'].replace({'ü': 'ue','Ü': 'Ue', 'ö': 'oe','Ö': 'Oe', 'ä': 'ae','Ä': 'Ae', 'ß': 'ss'}, regex=True)

#Nicht-String-Eingabe in String umwandeln
df['Titel_bereinigt'] = df['Titel_bereinigt'].astype(str)
df['Text_bereinigt'] = df['Text_bereinigt'].astype(str)

#Falsch übersetzte Woerter anpassen
df['Titel_bereinigt'] = df['Titel_bereinigt'].replace({'datum': 'date','ki':'ai','chatt':'chat','Kiai':'AI','gptuse':'GPT','Openaai':'Openai','openaai':'Openai'}, regex=True)
df['Text_bereinigt'] = df['Text_bereinigt'].replace({'datum': 'date','ki':'ai','chatt':'chat','Kiai':'AI','gptuse':'GPT','Openaai':'Openai','openaai':'Openai'}, regex=True)

#Extrahierte Daten, welche kein wirklicher Teil des Nachrichtenartikels sind 
df['Titel_bereinigt'] = df['Titel_bereinigt'].replace({'href':'','https':'','wwwrndde':'','dpa':'','Der Spiegel':'','ZEIT':'','rnd':'','RND':'','FAZ':'','Tagesspiegel':'','Handelsblatt':'','Zerr':'','ZEER':'',' B ':'',' b ':''}, regex=True)
df['Text_bereinigt'] = df['Text_bereinigt'].replace({'href':'','https':'','wwwrndde':'','dpa':'','Der Spiegel':'','time online':'','ZEIT':'','rnd':'','RND':'','FAZ':'','Tagesspiegel':'','Handelsblatt':'','Zerr':'','ZEER':'',' B ':'',' b ':''}, regex=True)


In [97]:
#Entfernen von Stoppwoertern aus Überschrift und Inhalt

#Stoppwoerter und Tokenizer herunterladen
nltk.download('stopwords')
nltk.download('punkt')

#Deutsche Stoppwoerter laden
stop_words = set(stopwords.words('english'))

#Entfernen von Stoppwoertern
def remove_stop_words(text):
    #Tokenisierung des Textes
    word_tokens = word_tokenize(text)
    #Die Stoppwörter herausfiltern
    filtered_sentence = [word for word in word_tokens if not word.lower() in stop_words]
    #Rekonstruieren Sie den Text ohne Stoppwörter
    return ' '.join(filtered_sentence)

#Stoppwoerter aus den Spalten "Ueberschrift" und "Inhalt" entfernen
df['Titel_bereinigt'] = df['Titel_bereinigt'].apply(lambda x: remove_stop_words(x) if isinstance(x, str) else x)
df['Text_bereinigt'] = df['Text_bereinigt'].apply(lambda x: remove_stop_words(x) if isinstance(x, str) else x)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MarkE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MarkE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [98]:
#Pruefung der Bereinigung

#Pruefung der Daten
print(df.shape)
print(df.describe())
print(df.head())

Zeile = 1

#Drucken der Spalten "Text" und "Text_bereinigt" für Zeile 2
print("URSPRUNG: ",df.loc[Zeile, 'Text'])
print("____")
print("NACH BEREINIGUNG: ",df.loc[Zeile, 'Text_bereinigt'])

(2053, 12)
                               Datum  Anzahl Woerter Text    KI Anteil  \
count                           2053          2053.000000  2053.000000   
mean   2023-08-20 07:02:15.021919232           722.042864     0.037392   
min              2021-11-22 00:00:00           250.000000     0.001439   
25%              2023-04-20 00:00:00           398.000000     0.019608   
50%              2023-09-29 00:00:00           587.000000     0.035156   
75%              2024-01-20 00:00:00           860.000000     0.052533   
max              2024-04-30 00:00:00          4391.000000     0.139623   
std                              NaN           472.120061     0.022031   

       Anzahl KI Wörter  Einmalige KI Wörter  
count       2053.000000          2053.000000  
mean          13.004384             3.340964  
min            2.000000             2.000000  
25%            6.000000             2.000000  
50%           11.000000             3.000000  
75%           17.000000             4.00

# Filter von Artikel mit Bezug zur Kuenstliche Intelligenz

In [99]:
#AI Woerter basierend auf KI Monitor 2022: 

#artifical intelligence • autonomous driving • machine learning • natural language processing • neural net • robotics • image recognition • künstliche intelligenz • autonomes fahren • maschinelles lernen • neuronales netz • natürliche sprachverarbeitung • robotik • bilderkennung

#Definiere die Liste der Schluesselwoerter mit expliziten Wortgrenzen für kurze Wörter
keywords = [
    r'\bai\b',r'\bki\b', r'\bchatgpt\b', r'\bagi\b', r'\bgpt\b', r'\brobot\b', r'\bbard\b',
    r'\blanguage processing\b',r'\blanguage model\b', r'\bspeech processing\b', r'\bvoice processing\b', r'\brobotics\b', r'\bimage recognition\b',
    r'\bartificial intelligence\b', r'\bchat bot\b', r'\bartificial intelligent\b',  
    r'\bmachine learning\b', r'\bneural network\b', r'\bdeep learning\b', r'\bautonomous driving\b', r'\bneural net\b',
    r'\bchat gpt\b', r'\bchatbot\b', r'\bartificial superintelligence\b',r'\bvoice model\b'
]

#Schluesselwort-Mapping, das angibt, wie Schluesselwoerter zusammengefasst werden sollen
keyword_mapping = {
    r'\bai\b': 'ai',
    r'\bki\b': 'ai',
    r'\bagi\b': 'ai',
    r'\bartificial intelligent\b': 'artificial intelligence',
    r'\bartificial intelligence\b': 'artificial intelligence',
    r'\bartificial superintelligence\b': 'artificial intelligence',
    r'\bchatgpt\b': 'chatgpt',
    r'\bchat gpt\b': 'chatgpt',
    r'\bchat bot\b': 'chatbot',
    r'\bchatbot\b': 'chatbot',
    r'\bgpt\b': 'gpt',
    r'\brobot\b': 'robotics',
    r'\brobotics\b': 'robotics',
    r'\bbard\b': 'bard',
    r'\blanguage processing\b': 'language processing',
    r'\blanguage model\b': 'language processing',
    r'\bspeech processing\b': 'voice processing',
    r'\bvoice processing\b': 'voice processing',
    r'\bvoice model\b': 'voice processing',
    r'\bimage recognition\b': 'image recognition',
    r'\bmachine learning\b': 'machine learning',
    r'\bneural network\b': 'neural network',
    r'\bneural net\b': 'neural network',
    r'\bdeep learning\b': 'deep learning',
    r'\bautonomous driving\b': 'autonomous driving',
}

#Funktion zur Berechnung der Term Frequency (TF) und zur Zählung der Schlüsselwoerter
def calculate_tf_and_details(document):
    words = document.lower()
    doc_length = len(words.split())
    word_counts = Counter()
    
    #Suche jedes Schlüsselwort mithilfe von regulären Ausdrücken und wende das Mapping an
    for regex, mapped_word in keyword_mapping.items():
        found_words = re.findall(regex, words)
        if found_words:
            word_counts[mapped_word] += len(found_words)
    
    keyword_count = sum(word_counts.values())
    tf = keyword_count / doc_length if doc_length > 0 else 0
    
    #Formatierung der zusammengefassten Schlüsselwörter mit Häufigkeiten
    ai_words = ', '.join(f"{keyword} ({count})" for keyword, count in word_counts.items())
    unique_count = len(word_counts)
    
    return tf, keyword_count, unique_count, ai_words

#Hilfsfunktion, um die Ergebnisse auf die neuen Spalten aufzuteilen
def apply_tf_and_details(df, column_name):
    results = df[column_name].apply(calculate_tf_and_details)
    df['KI Anteil'], df['Anzahl KI Wörter'], df['Einmalige KI Wörter'], df['KI Wörter'] = zip(*results)

#Wende die Funktion auf die Spalte "Text_bereinigt" an und füge die Ergebnisse dem DataFrame hinzu
apply_tf_and_details(df, 'Text_bereinigt')

#Zeige die ersten 20 Zeilen der neuen Spalten zusammen mit den Überschriften an
print(df.loc[:19, ['Titel', 'KI Anteil', 'Anzahl KI Wörter','Einmalige KI Wörter','KI Wörter']])

                                                Titel  KI Anteil  \
0   KI: Warum wir nicht mit Roboter-Autos vollauto...   0.042115   
1   Microsoft: Sind Jahrzehnte von einer künstlich...   0.058442   
2   Künstliche Intelligenz: Europa muss von neuen ...   0.031593   
4   Was die Superintelligenz-KI anrichten könnte, ...   0.063241   
5   Künstliche Intelligenz: Wie man sich seine Pro...   0.046512   
6   Attacke auf Chinas KI-Hoffnung: Leerverkäufer ...   0.030973   
9   KI-Fähigkeiten am digitalen Arbeitsmarkt: Präm...   0.069212   
10   Schicksalstage der KI-Verordnung: drei Szenarien   0.047170   
11  KI-Unternehmen Anthropic: Abtrünnige von OpenA...   0.059908   
19  EZB-Analyse: KI bislang kein Jobkiller für eur...   0.061111   

    Anzahl KI Wörter  Einmalige KI Wörter  \
0                 47                    6   
1                  9                    3   
2                 23                    3   
4                 16                    4   
5                 26      

In [100]:
#Filter Zeilen mit KI: Mindestens 2 einmalige Schlagwoerter im Text
df = df[df["Einmalige KI Wörter"] >= 2]

In [101]:
# Save the dataframe in a csv file
df.to_csv('4_Daten_bereinigt.csv', index=False)